In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
hidden_size = 128
input_width = 28
input_height = 28
output_size = 10

In [3]:
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28], name="X")
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name="Y")

W = tf.Variable(tf.random_normal(dtype=tf.float32, shape=[hidden_size, 10]), name="W")
b = tf.Variable(tf.zeros(shape=[10]), name="b")
cell = tf.contrib.rnn.BasicRNNCell(hidden_size)
output, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

output = tf.transpose(a=output, perm=[1, 0, 2], name="output")
output = output[-1]

model = tf.add(tf.matmul(output, W), b, name="model")

In [4]:
#model.shape

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))#cast ==> 소숫점 떼 주는 역할

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model), name="cost")
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [5]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)
#원인모를 gpu 문제 때문에 첨가한 line ↓ 소스코드 영향 無
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
#config=tf.ConfigProto(gpu_options=gpu_options) 도 원인모를 gpu 문제때문에 첨가한 line ↓ 소스코드 영향 無
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(5):
        total_cost = 0

        for i in range(total_batch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            x_batch = x_batch.reshape(batch_size, 28, 28)
            opt, cost_val = sess.run([optimizer, cost], feed_dict={X : x_batch, Y : y_batch})
            total_cost = total_cost + cost_val

        print("epoch : ", (epoch+1), "cost : ", (total_cost / total_batch))
        
    
    test_batch_size = len(mnist.test.images)
    test_xs = mnist.test.images.reshape(test_batch_size, 28, 28)
    test_ys = mnist.test.labels

    print('정확도:', sess.run(accuracy,feed_dict={X: test_xs, Y: test_ys}))

epoch :  1 cost :  0.527661050368439
epoch :  2 cost :  0.2351197329502214
epoch :  3 cost :  0.19016067160124128
epoch :  4 cost :  0.16152295913208614
epoch :  5 cost :  0.1458551808209582
정확도: 0.9621
